In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline 

In [47]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from common import rotate_to_sector0,uv2sectoru0v0

In [53]:
df_geom = pd.read_csv('../data/geometry_bestguess_v2.hgcal.txt', sep=' ')
df_constr = pd.read_csv('../data/220712_strobbe_elinks-mapping.csv')
showermax_layers = [7,9,11,13]
hcal_start = 27
elinks_showermax = 4
elinks_regular = 2
# STC: assuming STC4 5E+4M in HDM and STC16 in LDM
elinks_stc16 = 2
elinks_stc4 = 5

In [54]:
print(df_geom.head())
print(df_constr.head())

   plane  u  v itype       x0       y0  irot  nvertices     vx_0     vy_0  \
0      1  1  2   dIe   -0.002  290.016     4          5  -83.718  338.396   
1      1  1  3    FI  -83.725  435.025     2          6    0.047  386.682   
2      1  1  4    FI -167.438  580.028     2          6  -83.763  531.718   
3      1  1  5    FI -251.161  725.037     2          6 -167.487  676.705   
4      1  1  6    FO -334.883  870.046     2          6 -251.210  821.691   

   ...  trigPp0_type            dataPp1           trigPp1  dataPp1_type  \
0  ...             B  PP1(CE-)11_FO_1_2  PP1(CE-)9_FO_2_3            Ap   
1  ...             B  PP1(CE-)11_FO_1_2  PP1(CE-)9_FO_2_3            Ap   
2  ...             B  PP1(CE-)11_FO_1_2  PP1(CE-)9_FO_2_3            Ap   
3  ...             B  PP1(CE-)11_FO_1_2  PP1(CE-)9_FO_2_3            Ap   
4  ...             A  PP1(CE-)11_FO_1_1  PP1(CE-)9_FO_2_1            Ap   

   trigPp1_type         dataPp2      DAQ  engine_trig_fibres  \
0            Ap  PPFO(

In [55]:
df_geom = df_geom[['plane','u','v','trigLinks','itype']]
df_geom[['sector60', 'u0', 'v0']] = df_geom.apply(uv2sectoru0v0, axis=1)
df_constr = df_constr.set_index(['layer', 'u', 'v'])

In [59]:
df_geom[(df_geom.sector60!=0) & (df_geom.plane<hcal_start)]

,plane,u,v,trigLinks,itype,sector60,u0,v0
0,1,1,2,5.0,dIe,1,2,1
1,1,1,3,8.0,FI,1,3,2
2,1,1,4,5.0,FI,1,4,3
3,1,1,5,4.0,FI,1,5,4
4,1,1,6,3.0,FO,1,6,5
...,...,...,...,...,...,...,...,...
2659,26,8,8,0.0,FO,1,8,0
2660,26,8,9,0.0,FO,1,9,1
2661,26,8,10,0.0,bOe,1,10,2
2671,26,9,9,0.0,FOe,1,9,0


In [61]:
def get_constr_bcstc(row):
    layer = int(row.plane)
    u = int(row.u)
    v = int(row.v)
    # Rotate to 60deg sector0 if in CE-E
    u0 = int(row.u0) if layer<hcal_start else u
    v0 = int(row.v0) if layer<hcal_start else v
    itype = row.itype
    triangle = itype[0]=='c'
    hdm = 'I' in itype
    bcstc_elinks = 0
    constr_elinks = 999
    bcstc_elinks = elinks_showermax if layer in showermax_layers else elinks_regular
    # no trigger links in even layers
    if layer%2==0 and layer<hcal_start: bcstc_elinks = 0 
    # STC elinks: STC4 in HDM and STC16 in LDM
    if layer>=hcal_start:
        if hdm:
            bcstc_elinks = elinks_stc4
        else:
            bcstc_elinks = elinks_stc16
    layer_tmp = min(layer, hcal_start) # all HCAL layers have the same constraints
    if df_constr.index.isin([(layer_tmp, u0, v0)]).any():
        constr_elinks = int(df_constr.loc[(layer_tmp,u0,v0)])
    # No trigger elinks in triangles/three-partials
    if triangle: constr_elinks = 0
    return min(bcstc_elinks, constr_elinks)

In [62]:
df_geom['trigLinksBCSTC'] = df_geom.apply(get_constr_bcstc, axis=1)

In [63]:
df_geom['ok'] = df_geom['trigLinks']>=df_geom['trigLinksBCSTC']

In [64]:
df_geom.query('ok==False')

,plane,u,v,trigLinks,itype,sector60,u0,v0,trigLinksBCSTC,ok
